In [1]:
import numpy as np
import pandas as pd
import scipy
import scipy.linalg
from scipy.stats import norm

In [2]:
price = pd.read_excel('k10_historical_price.xlsx')
rets = np.log(price/price.shift(1))
rets.dropna(inplace=True)

In [3]:
# Cholesky Decomposition
corr_ret = np.array(rets.corr())
C = scipy.array(corr_ret)
L = scipy.linalg.cholesky(C, lower = True)
U = scipy.linalg.cholesky(C, lower = False)

In [4]:
mean = rets.mean()
std = rets.std()

In [5]:
def MonteCarloRet(mu, sigma, lowerMatrix, number=1000):    
    MC_rets = []
    for n in range(number):
        
        # 주어진 주가수익률의 평균과 표준편차를 가지는 정규분포로부터 랜덤 샘플 추출
        standard_ranRet = np.array([norm(mu[i],sigma[i]).rvs(1)[0] for i in range(len(mu))])
        
        # 원 샘플의 상관관계를 가지게 하기 위해 Cholesky decomposition에서 나온 lower triangular matrix L을 곱함.
        corr_renRet = lowerMatrix.dot(standard_ranRet)
        corr_renRetlist = [corr_renRet[i] for i in range(len(mu))]
        MC_rets.append(corr_renRetlist)
    MC_rets = np.array(MC_rets)
    return MC_rets

def VaR(weights, returns):
    MC_portfolio_rets = (returns * weights).sum(axis=1)
    VaR_per_price = -np.quantile(MC_portfolio_rets, 0.05)
    return VaR_per_price

In [6]:
MC_rets = MonteCarloRet(mean, std, L)

In [7]:
weight = [0.1]*10
VaR(weight, MC_rets)

0.01690292822584758

In [8]:
# Minimum VaR portfolio
import scipy.optimize as sco

def optimizeVaR(weights):
    return VaR(weights, MC_rets)

cons = ({'type':'eq', 'fun':lambda x: np.sum(x)-1})
bnds = tuple((0,1) for x in range(10))

opts = sco.minimize(optimizeVaR, weight, method='SLSQP', bounds=bnds, constraints=cons)
optimum_weight = opts.x
optimum_weight

array([1.53191061e-03, 3.14467975e-02, 1.36090984e-01, 7.31989569e-19,
       2.96329707e-02, 1.22033021e-01, 2.74998344e-01, 1.50133741e-01,
       1.35985163e-01, 1.18147068e-01])

In [9]:
pf_ret = (mean * optimum_weight * 252).sum()
pf_std = np.sqrt(np.dot(optimum_weight.T, np.dot(rets.cov()*252, optimum_weight)))
print("Expected Return: {}".format(pf_ret.round(3)))
print("Expected Volatility: {}".format(pf_std.round(3)))
print("Expected Sharpe Ratio: {}".format((pf_ret/pf_std).round(3)))

Expected Return: 0.148
Expected Volatility: 0.117
Expected Sharpe Ratio: 1.268
